In [1]:
import torch
from torch import utils
from torchvision import datasets, transforms

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
path='./MNIST_data'

#### Смотрим на данные

In [3]:
train_data = datasets.MNIST(path, train=True, download=True, transform=transforms.ToTensor())
train_data


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./MNIST_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
# параметры датасета
len_train = len(train_data)
print(f"кол-во элементов в датасете: {len_train}")
print(f"тип элемента:  {type(train_data[0])}")
print(f"длинна элемента:  {len(train_data[0])}")

кол-во элементов в датасете: 60000
тип элемента:  <class 'tuple'>
длинна элемента:  2


In [5]:
# посмотрим на содержимое 
item = train_data[0][0] 
print(item.shape)
print(f"min: {item.min()} max: {item.max()}")
train_data[0][0]

torch.Size([1, 28, 28])
min: 0.0 max: 1.0


tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

#### Вычисления и преобразования

Судя по всему, при загрузке, по умолчанию была произведена min-max нормализация
Наша задача сделать std нормализацию 
Для этого найдем среднее и стандартное отклонение

In [6]:
def get_train_mean_std(train_set):
    """
    Получаем тренировочный датасет
    Возвращаем кортеж (mean, std) 
    """
    # получаем и приводим к numpy данные из тренировочного датасета
    items_ = [train_set[i][0].numpy() for i in range(len(train_set))]
    mean_ = np.mean(items_)
    std_ = np.std(items_)
    return mean_, std_

In [7]:
# получим расчетные данные
k_mean, k_std = get_train_mean_std(train_data)
print(f"Mean = {k_mean}\nStd = {k_std}")

Mean = 0.13066062331199646
Std = 0.30810776352882385
CPU times: user 7.69 s, sys: 270 ms, total: 7.96 s
Wall time: 7.78 s


In [8]:
mnist_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((k_mean,), (k_std,))
                ])

In [9]:

# загружаем с модифицированной нормализацией
train_data_mod = datasets.MNIST(path, train=True, download=True, transform=mnist_transform)

In [10]:
# проверяем результат
mod_mean, mod_std = get_train_mean_std(train_data_mod)
print(f"Mean = {mod_mean.round(5)}\nStd = {mod_std.round(5)}")

Mean = -0.0
Std = 1.0
CPU times: user 11.5 s, sys: 290 ms, total: 11.8 s
Wall time: 11.6 s


### OK!
